In [209]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import KNNBasic,KNNWithMeans
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import train_test_split
from collections import defaultdict
import io
from surprise.model_selection import GridSearchCV
import csv
import os

In [213]:
df = pd.read_csv('C:/Users/arshi/Desktop/ArshiyaUSB/Fall2018/256/Project/CMPE256-Airbnb/Module1/output/rating_prediction_full.csv')

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800995 entries, 0 to 800994
Data columns (total 3 columns):
listing_id     800995 non-null int64
reviewer_id    800995 non-null int64
rating         800995 non-null int64
dtypes: int64(3)
memory usage: 18.3 MB


In [215]:
df.nunique()


listing_id      34839
reviewer_id    703051
rating              6
dtype: int64

Check count of reviews given by user

In [216]:
df.reviewer_id.value_counts()

197711       63
3973614      52
76641978     42
339131       40
2539165      36
101640211    34
39274139     33
563572       33
77978742     32
24667379     32
84558452     30
31402335     30
15355355     30
26152563     28
43382407     28
22585386     28
45139957     28
23545132     27
21726142     26
14438        26
39177007     26
76628146     25
95940868     24
28774703     24
320624       24
5429871      24
109699676    24
86126627     23
77352080     23
17387960     22
             ..
138197010     1
55263049      1
86734363      1
20978485      1
62925619      1
31466290      1
25180977      1
16790320      1
24167214      1
17100542      1
18064194      1
50631481      1
87983072      1
8647496       1
140727132     1
1272667       1
115565402     1
55329984      1
31683416      1
59318762      1
18620222      1
69407572      1
27468626      1
39865320      1
287600        1
1301325       1
82039628      1
73655114      1
66321225      1
41258800      1
Name: reviewer_id, Lengt

Check count of reviews every listings got

In [178]:
df.listing_id.value_counts()

903972      489
891117      474
903947      462
21218       394
26785       383
546383      359
31994       352
58059       351
5178        350
8168619     342
834190      339
467634      336
476983      332
10101135    328
9145202     326
27024       322
1093570     322
557018      320
63360       318
21220       311
847690      308
67208       306
151199      304
999248      298
7675781     298
4261693     297
394474      296
2284027     295
106363      295
1120700     295
           ... 
11548744      1
9282691       1
16072935      1
19589738      1
9959880       1
6233819       1
4838260       1
20811403      1
13410478      1
16381984      1
8742625       1
2744182       1
17517651      1
9944013       1
20933335      1
6688119       1
7827616       1
2365645       1
8014087       1
19751573      1
20662567      1
16441446      1
7922002       1
19163061      1
19117903      1
20571653      1
8814070       1
3323247       1
7365429       1
10163240      1
Name: listing_id, Length

In [217]:
df.rating.value_counts()

5    324276
4    212149
3    119025
0     81108
2     50666
1     13771
Name: rating, dtype: int64

Removing entries with rating 0.

In [218]:
df = df[df.rating !=0]

In [219]:
df.shape

(719887, 3)

As We are going to use collaborative filtering algorithm to predict ratings, we will limit data by considering listings which got atleast 10 reviews and users wo have rated more than 3 listings.
if there are m users and n listings, matrix size will be m*n which is very big.

In [220]:
counts = df['listing_id'].value_counts()
#print(counts)
df= df[df['listing_id'].isin(counts[counts >= 10].index)]

In [221]:
df.shape

(656216, 3)

In [222]:
counts1 = df['reviewer_id'].value_counts()
#print(counts1)
df= df[df['reviewer_id'].isin(counts1[counts1 > 3].index)]

In [223]:
df.describe()

,listing_id,reviewer_id,rating
count,2.307900e+04,2.307900e+04,23079.000000
mean,7.250671e+06,2.977601e+07,4.327917
std,5.770643e+06,3.303118e+07,0.902624
min,2.595000e+03,1.000000e+00,1.000000
25%,1.710644e+06,3.995723e+06,4.000000
50%,6.367973e+06,1.736788e+07,5.000000
75%,1.233312e+07,4.488507e+07,5.000000
max,2.060036e+07,1.446207e+08,5.000000


In [224]:
df.head()

,listing_id,reviewer_id,rating
18,9452127,56075148,5
76,6078066,51369115,5
97,6078066,38699430,5
101,6078066,5273094,5
146,10120476,5353500,4


In [225]:
df1 = df[['reviewer_id','listing_id','rating' ]]
df1.head()
print(df1.nunique())

reviewer_id    4320
listing_id     8777
rating            5
dtype: int64


In [189]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df1[['reviewer_id', 'listing_id', 'rating']], reader)

data1 = data

In [190]:

raw_ratings = data.raw_ratings

# shuffle ratings if you want
#random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005,0.1],
              'reg_all': [0.4, 0.6], 'n_factors':[100,500]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)
print('best RMSE score')
print(grid_search.best_score['rmse'])

Grid Search...
best RMSE score
0.886203998266


In [191]:
print('combination of parameters that gave the best RMSE score')
print(grid_search.best_params['rmse'])
algo = grid_search.best_estimator['rmse']


combination of parameters that gave the best RMSE score
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 100}


In [195]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
testset= trainset.build_testset()
predictions = algo.test(testset)
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions,verbose=True)
print('len(predictions)')
print(len(predictions))

Biased accuracy on A,   RMSE: 0.7483
len(predictions)
20771


In [196]:
# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)
print('len(predictions)')
print(len(predictions))

Unbiased accuracy on B, RMSE: 0.8505
len(predictions)
2308


In [197]:
# define a cross-validation iterator
kf = KFold(n_splits=7)
#algo = SVD(n_factors=500, n_epochs=5, lr_all=0.1)

for trainset, testset in kf.split(data1):
    # train and test algorithm.
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


RMSE: 0.8767
RMSE: 0.8668
RMSE: 0.8983
RMSE: 0.8842
RMSE: 0.8997
RMSE: 0.8707
RMSE: 0.8837


Build anti test set for predicting top 3 ratings for user

In [206]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

testset= trainset.build_anti_testset()
predictionsAll = algo.test(testset)
print('Accuracy on whole data set,', end='   ')
accuracy.rmse(predictionsAll,verbose=True)
print('len(predictions)')
print(len(predictionsAll))

Accuracy on whole data set,   RMSE: 0.1473
len(predictions)
34131259


In [200]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]    
    return top_recs
print('getting top 3 recommendations')




getting top 3 recommendations


In [ ]:
top3_recommendations = get_top3_recommendations(predictionsAll,3)

print('Total predictions calculated are %d' % len(predictionsAll))


Writing top 3 recommendations calculated to csv file

In [202]:
dfo = pd.DataFrame(columns=['UserId', 'Recommended Listing,Rating'])
i=0;
for uid, user_ratings in top3_recommendations.items():
    #print(uid, top3_recommendations[uid])
    row = [uid, top3_recommendations[uid]]
    dfo.loc[i] = row
    i=i+1
dfo.to_csv('submissionF.csv', index = False)

print("wrote Recommendations for each user in csv file")     
        


done
